In [54]:
import requests
from urllib.parse import urlencode
import base64
import webbrowser


In [55]:

client_id = "a6eea1959e344ead823106e86277724e"
client_secret = "479f3fabe040437eb3c4be2e7c472bad"

auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "user-read-recently-played"
}

webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

True

In [56]:
code = "AQCLXqbkTCZnBwY5Z5wnU0-dJMFWW0cwUja1vE-L6OTYhFEL11DWszN-ZAoqCtgKUpgopGy1s_1o-Arr-yCOQunoBxlFF8mvghqJEGULgfSCEHe7o9hvGnHMWdefDITqBhZrSt9U0FvVFuCfhM-MySghsXxne_gGlRrIKwG7ErWoEhz0pJZUum2s6mLTuIs-4gAhz1_ffC275BXMmQ"

In [57]:

encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)

In [58]:
r

<Response [200]>

In [59]:
token = r.json()["access_token"]

print(token)

BQDFoEoc5tVbyHS6BIWg-llFNg0B8LjrrMA92VFo1StOxCSNFlZLG-DZcTlAXZ2ZsWr9Fy5o7imwpE1MDp4nUOOJCQYn4enpH-zi2oDcJPMGeQzFYgI52ycJowCycntgWPPh8wVkj8ImMa_uSJ9sRPgIPZuaA7X40VsqNe-xmjhLWiihtuNQP0NBx__ZsrLkcZRC0E6Qxzc


# EXTRACT

In [60]:
import pandas as pd 
import requests
from datetime import datetime
import datetime


USER_ID = "Mankirat"
TOKEN = token


# Creating an function to be used in other pyrhon files
def return_dataframe(): 
    input_variables = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }
     
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=2)
    yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    # Download all songs you've listened to "after yesterday", which means in the last 24 hours      
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = input_variables)

    data = r.json()
    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []
    

    # Extracting only the relevant bits of data from the json object      
    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])
        
    # Prepare a dictionary in order to turn it into a pandas dataframe below       
    song_dict = {
        "song_name" : song_names,
        "artist_name": artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps
    }
    song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])
    return song_df

# TRANSFORM

In [61]:
import pandas as pd 

# Set of Data Quality Checks Needed to Perform Before Loading
def Data_Quality(load_df):
    #Checking Whether the DataFrame is empty
    if load_df.empty:
        print('No Songs Extracted')
        return False
    
    #Enforcing Primary keys since we don't need duplicates
    if pd.Series(load_df['played_at']).is_unique:
       pass
    else:
        #The Reason for using exception is to immediately terminate the program and avoid further processing
        raise Exception("Primary Key Exception,Data Might Contain duplicates")
    
    #Checking for Nulls in our data frame 
    if load_df.isnull().values.any():
        raise Exception("Null values found")

# Writing some Transformation Queries to get the count of artist
def Transform_df(load_df):

    #Applying transformation logic
    Transformed_df=load_df.groupby(['timestamp','artist_name'],as_index = False).count()
    Transformed_df.rename(columns ={'played_at':'count'}, inplace=True)

    #Creating a Primary Key based on Timestamp and artist name
    Transformed_df["ID"] = Transformed_df['timestamp'].astype(str) +"-"+ Transformed_df["artist_name"]

    return Transformed_df[['ID','timestamp','artist_name','count']]

if __name__ == "__main__":

    #Importing the songs_df from the Extract.py
    load_df=return_dataframe()
    Data_Quality(load_df)
    #calling the transformation
    Transformed_df=Transform_df(load_df)    
    print(Transformed_df)
    

                          ID   timestamp    artist_name  count
0          2023-07-14-DIVINE  2023-07-14         DIVINE      1
1     2023-07-14-Don Toliver  2023-07-14    Don Toliver      1
2      2023-07-14-Elley Duhé  2023-07-14     Elley Duhé      1
3           2023-07-14-Fateh  2023-07-14          Fateh      3
4           2023-07-14-Gunna  2023-07-14          Gunna      2
5            2023-07-14-Jaan  2023-07-14           Jaan      2
6           2023-07-14-Lithe  2023-07-14          Lithe      1
7        2023-07-14-Mr.Dhatt  2023-07-14       Mr.Dhatt      1
8   2023-07-14-Nipsey Hussle  2023-07-14  Nipsey Hussle      2
9      2023-07-14-The Weeknd  2023-07-14     The Weeknd      1
10      2023-07-14-Tom Odell  2023-07-14      Tom Odell      1
11        2023-07-14-Tympana  2023-07-14        Tympana      1
12    2023-07-14-Wazir Patar  2023-07-14    Wazir Patar      1
13   2023-07-14-XXXTENTACION  2023-07-14   XXXTENTACION      1
14       2023-07-14-omgkirby  2023-07-14       omgkirby

# LOAD

In [66]:
import sqlalchemy
import pandas as pd 
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"

if __name__ == "__main__":

#Importing the songs_df from the Extract.py
    load_df=return_dataframe()
    if(Data_Quality(load_df) == False):
        raise ("Failed at Data Validation")
    Transformed_df=Transform_df(load_df)
    #The Two Data Frame that need to be Loaded in to the DataBase

#Loading into Database
    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('my_played_tracks.sqlite')
    cursor = conn.cursor()

    #SQL Query to Create Played Songs
    sql_query_1 = '''
    CREATE TABLE IF NOT EXISTS my_played_tracks(
        song_name VARCHAR(200),
        artist_name VARCHAR(200),
        played_at VARCHAR(200),
        timestamp VARCHAR(200),
        CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
    )
    '''
    #SQL Query to Create Most Listened Artist
    sql_query_2 = '''
    CREATE TABLE IF NOT EXISTS fav_artist(
        timestamp VARCHAR(200),
        ID VARCHAR(200),
        artist_name VARCHAR(200),
        count VARCHAR(200),
        CONSTRAINT primary_key_constraint PRIMARY KEY (ID)
    )
    '''
    cursor.execute(sql_query_1)
    cursor.execute(sql_query_2)
    print("Opened database successfully")

    #We need to only Append New Data to avoid duplicates
    try:
        load_df.to_sql("my_played_tracks", engine, index=False, if_exists='append')
    except:
        print("Data already exists in the database")
    try:
        Transformed_df.to_sql("fav_artist", engine, index=False, if_exists='append')
    except:
        print("Data already exists in the database2")

    #cursor.execute('DROP TABLE my_played_tracks')
    #cursor.execute('DROP TABLE fav_artist')

    conn.close()
    print("Close database successfully")

Opened database successfully
Data already exists in the database
Data already exists in the database2
Close database successfully
